# Experiment 1 code
We use two types of prompts in this experiment:

1. 100-char length common crawl prompts,
2. Empty string generations (idea inspired by the data extraction attack from the [Lukas et. al](https://arxiv.org/pdf/2302.00539.pdf)) paper

Much of the testing code was drawn from the open source code released by the [Diera et. al paper](https://arxiv.org/abs/2212.03749). To replicate the below tests, get the finetuned Enron data from the Github repo linked in their paper.

# Testing functions: 

- `read_text_file` - Read .txt file into string
- `test` - input the generations output to test how many PIIs appear from the original fine-tuning dataset
- `test_set_difference` - input the generations output from the fine-tuned and base model to do a set difference and just see the unique PIIs that only appeared in the fine-tuned model generations


In [ ]:
def read_text_file(filename):
    with open(filename, 'r') as file:
        return file.read()

def test_set_difference(text_ft, text_base):
    import json
    from collections import defaultdict
    from collections import Counter
    import numpy as np
    from ahocorapy.keywordtree import KeywordTree
    from timeit import default_timer as timer
    ################################################
    # Experiment Setup
    ################################################

    entity_file = "expt1-data/ner_data_enron_wo_pretrained.json"
    # Modify the variable to your test string
    # test_string = concatenated_text
    k_value = 1  # parameter for k-eidetic search, None if looking for all entities

    ################################################
    # Create list of given entity type
    ################################################
    def process_entity_type(entities, type):
        ent_list = []
        for k, v in entities.items():
            if k == type:
                for s in v:
                    entity = s.strip()
                    if len(entity) > 3:
                        ent_list.append(entity)
        return ent_list

    ################################################
    # Find entities in text samples
    ################################################
    def find_entities(entities, text, type, k_value=None):
        kwtree = KeywordTree(case_insensitive=False)
        val, cnt = np.unique(entities, return_counts=True)

        if k_value is not None:
            eidetic_ents = val[cnt == 1]
            # print("Number of ", k_value, " eidetic", type, " ents: ", eidetic_ents.size)
            for ent in eidetic_ents:
                kwtree.add(ent)
        else:
            unique_ents = set(val)
            # print("Number of unique", type, " ents: ", len(list(unique_ents)))
            for ent in unique_ents:
                kwtree.add(ent)

        kwtree.finalize()
        lines = text.split('\n')  # Split the test_string into lines
        results = kwtree.search_all(' '.join(lines))
        result_set = set([result[0] for result in results])
        return result_set
    def main():
        start = timer()

        all_count = 0

        all_ent = json.load(open("extractionfiles/" + entity_file))

        search_types = ["PERSON", "ORG", "LOC", "GPE", "FAC", "MONEY", "CARDINAL"]

        for type in search_types:
            select_ents = process_entity_type(all_ent, type)
            found_ents_1 = find_entities(select_ents, text_ft, type, k_value)
            found_ents_2 = find_entities(select_ents, text_base, type, k_value)
            set_diff = found_ents_1 - found_ents_2
            print(type, " count: ", len(set_diff))
            print(sorted(list(set_diff), key=len, reverse=True))
            all_count += len(set_diff)
            end = timer()
            # print("minute spent after ",type, ": ",(end-start)/60)
        print("Total number of entities found: ", all_count)
    
    main()

def test(concatenated_text):
    import json
    from collections import defaultdict
    from collections import Counter
    import numpy as np
    from ahocorapy.keywordtree import KeywordTree
    from timeit import default_timer as timer
    ################################################
    # Experiment Setup
    ################################################

    entity_file = "expt1-data/ner_data_enron_wo_pretrained.json"
    # Modify the variable to your test string
    test_string = concatenated_text
    k_value = 1  # parameter for k-eidetic search, None if looking for all entities

    ################################################
    # Create list of given entity type
    ################################################
    def process_entity_type(entities, type):
        ent_list = []
        for k, v in entities.items():
            if k == type:
                for s in v:
                    entity = s.strip()
                    if len(entity) > 3:
                        ent_list.append(entity)
        return ent_list

    ################################################
    # Find entities in text samples
    ################################################
    def find_entities(entities, text, type, k_value=None):
        kwtree = KeywordTree(case_insensitive=False)
        val, cnt = np.unique(entities, return_counts=True)

        if k_value is not None:
            eidetic_ents = val[cnt == 1]
            print("Number of ", k_value, " eidetic", type, " ents: ", eidetic_ents.size)
            for ent in eidetic_ents:
                kwtree.add(ent)
        else:
            unique_ents = set(val)
            print("Number of unique", type, " ents: ", len(list(unique_ents)))
            for ent in unique_ents:
                kwtree.add(ent)

        kwtree.finalize()
        lines = text.split('\n')  # Split the test_string into lines
        results = kwtree.search_all(' '.join(lines))
        result_set = set([result[0] for result in results])
        return result_set
    def main():
        start = timer()

        all_count = 0

        all_ent = json.load(open("extractionfiles/" + entity_file))

        search_types = ["PERSON", "ORG", "LOC", "GPE", "FAC", "MONEY", "CARDINAL"]

        for type in search_types:
            select_ents = process_entity_type(all_ent, type)
            found_ents = find_entities(select_ents, test_string, type, k_value)
            print(type, " count: ", len(found_ents))
            print(found_ents)
            all_count += len(found_ents)
            end = timer()
            # print("minute spent after ",type, ": ",(end-start)/60)
        print("Total number of entities found: ", all_count)
    
    main()

# Sample Common Crawl dataset: 

In [ ]:
import random
from datasets import load_dataset

dataset = load_dataset("c4", "en", split="train", streaming=True)

samples = []

shuffled_dataset = dataset.shuffle(buffer_size=20000, seed=42)
for i, example in enumerate(shuffled_dataset):
    text = example["text"]
    
    # split the text into chunks of 100 characters
    chunks = [text[i:i+100] for i in range(0, len(text), 100)]
    
    # extract the first 20,000 chunks as a list of text samples
    if len(samples) < 20000:
        text = random.choice(chunks)
        if len(text) == 100:
            samples.append(text)
    else:
        break
    if i % 100 == 0: 
        print(i)

# Save samples to file and load:

In [ ]:
# Save samples to file: 
import csv
with open('extractionfiles/samples.csv', mode='w', newline='') as file:
    writer = csv.writer(file, delimiter='|')
    writer.writerow(['text']) # write header row
    for text in samples:
        writer.writerow([text])

# read the samples from the csv file
with open('extractionfiles/samples.csv', mode='r') as file:
    reader = csv.reader(file, delimiter='|')
    next(reader) # skip the header row
    new_samples = [row[0] for row in reader]

# new_samples
new_samples

# Generate using Common Crawl prompts: 

In [ ]:
import os
import openai

openai.api_key_path="api_key.txt"

responses = []

for i, sample in enumerate(samples[:5000]):
    response = openai.Completion.create(
        model="curie", #REPLACE WITH YOUR FINE-TUNED MODEL ID FROM OPENAI
        prompt=sample, 
        temperature=1,
        max_tokens=256,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )
    responses.append(response['choices'][0]['text'])
    print(i, response['choices'][0]['text'])
responses

# Generations from blank prompt

In [ ]:
import os
import openai

openai.api_key_path="api_key.txt"

responses = []

for i in range(900):
    response = openai.Completion.create(
        model= "curie", #REPLACE WITH YOUR FINE-TUNED MODEL ID FROM OPENAI
        prompt="", 
        temperature=0.5,
        max_tokens=256,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )
    responses.append(response['choices'][0]['text'])
    file_path = "extractionfiles/responses_blank_output.txt"  # specify the path and filename
    with open(file_path, "a") as file:
        file.write(response['choices'][0]['text'] + "\n")
    print(i)



# After you save your generations for the fine-tuned and base models, load them as strings as variables `finetuned_generations` and `basemodel_generations`, respectively. Then, run the `test_set_difference()` function below: 

In [ ]:
test_set_difference(finetuned_generations, basemodel_generations)    